# Mapping labels to IRS normalized proteomics results

### Import libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

import sys
import os

### Import normalized data

In [2]:
ori_df = pd.read_csv('../Intermediate data/IRS_norm_data_no_labels_sorted.csv')
ori_df = ori_df.rename(columns={'Unnamed: 0':'fasta_header'})
ori_df.head(50)

,fasta_header,BL11,BL12,BL21,BL22,CF11,CF12,CF21,CF22,POOL1,...,POOL3,POOL4,S111,S112,S121,S122,S711,S712,S721,S722
0,CONT_011|gi|136425|sp|P00760|TRYP_BOVIN (+1),3.834882e+06,3.910692e+06,3.873719e+06,3.901555e+06,3.731300e+06,4.510965e+06,4.588062e+06,4.236974e+06,3.904010e+06,...,4.161065e+06,3.962332e+06,3.785124e+06,3.831308e+06,4.691136e+06,4.209506e+06,3.870142e+06,3.996458e+06,4.224602e+06,5.063368e+06
1,CONT_060|gi|542923|pir||S37780 (+2),5.768847e+04,6.916108e+04,4.499092e+04,4.547218e+04,5.129144e+04,5.492603e+04,8.371912e+04,6.586420e+04,5.884843e+04,...,5.798025e+04,5.906861e+04,6.124590e+04,6.795917e+04,6.555273e+04,7.275093e+04,4.121419e+04,4.759626e+04,3.299722e+04,3.103007e+04
2,CONT_068|gi|1082558|pir||S41161 (+1),6.199343e+06,7.064887e+06,6.312329e+06,6.931382e+06,5.935896e+06,6.075726e+06,6.754209e+06,1.218231e+07,6.828453e+06,...,6.827672e+06,7.112692e+06,4.897239e+06,7.263064e+06,6.779768e+06,6.442336e+06,4.001778e+06,8.255296e+06,8.388862e+06,8.690249e+06
3,CONT_072|gi|547754|sp|P35908|K22E_HUMAN (+1),1.580685e+06,1.583641e+06,1.860521e+06,2.163102e+06,1.613313e+06,1.448806e+06,1.885191e+06,1.723407e+06,1.812941e+06,...,1.868804e+06,1.834231e+06,8.753931e+05,2.208658e+06,1.820206e+06,1.493089e+06,1.374815e+06,2.530316e+06,2.460016e+06,2.744847e+06
4,CONT_089|gi|71536|pir||KRHU2,1.919952e+06,2.442960e+06,2.154624e+06,2.331342e+06,2.301886e+06,2.177643e+06,2.417075e+06,4.278359e+06,2.414614e+06,...,2.422754e+06,2.404175e+06,1.594131e+06,2.433767e+06,2.269867e+06,2.144572e+06,1.557455e+06,2.699832e+06,3.269444e+06,3.409155e+06
5,CONT_092|gi|71528|pir||KRHU0 (+2),3.971203e+06,4.112515e+06,4.550645e+06,5.087362e+06,4.503944e+06,4.189552e+06,4.709996e+06,5.307843e+06,4.811199e+06,...,4.856297e+06,4.939114e+06,2.985942e+06,5.573998e+06,5.115626e+06,4.021971e+06,4.122821e+06,6.309237e+06,6.940652e+06,6.734369e+06
6,EXTRA_0117 (+1),1.440225e+07,1.976366e+07,1.777267e+07,1.323484e+07,1.061200e+07,1.441863e+07,1.598302e+07,1.290840e+07,3.680239e+07,...,3.594773e+07,3.689753e+07,1.271964e+07,1.192148e+07,1.637461e+07,1.248008e+07,1.228182e+08,1.244550e+08,1.007215e+08,1.100053e+08
7,EXTRA_0118,4.354697e+08,4.574848e+08,4.548247e+08,4.164842e+08,3.489221e+08,4.174575e+08,3.975685e+08,3.708042e+08,8.357021e+08,...,8.523579e+08,8.453671e+08,1.744299e+09,1.499142e+09,1.547300e+09,1.508134e+09,1.096133e+09,1.057119e+09,1.002331e+09,1.096688e+09
8,EXTRA_0119,1.539789e+08,2.000820e+08,1.979865e+08,1.596371e+08,1.080412e+08,2.495407e+08,1.974197e+08,1.836982e+08,8.359798e+08,...,8.565869e+08,8.412190e+08,2.061747e+09,1.934594e+09,1.918482e+09,1.835840e+09,1.233054e+09,1.280986e+09,1.098762e+09,1.286722e+09
9,EXTRA_0121,1.500844e+07,1.761701e+07,1.691416e+07,1.469569e+07,1.207333e+07,1.455480e+07,1.614900e+07,1.252156e+07,3.134059e+07,...,3.051643e+07,3.136487e+07,1.166578e+07,1.177868e+07,1.510703e+07,1.237014e+07,8.915036e+07,9.935692e+07,8.253212e+07,8.880221e+07


### Extract accession numbers from UNIPROT Identifiers
##### The labels come from the FASTA headers included in the FASTA database UP000002032 for Escherichia coli (strain B / BL21-DE3) merged with the possible contamination and exogenously expressed protein sequences. These FASTA files are included in the Initial filesdirectory.

##### Accession numbers are pulled from proteins in the database and contamination/ exogenously expressed proteins are flagged

In [4]:
def pull_acc(accession):
    if 'REV' in accession:
        return 'REV'
    elif 'CONT' in accession:
        return 'cont'
    elif 'EXTRA' in accession:
        return 'extra'
    elif 'REV' in accession:
        return 'REV'
    elif'|' in accession:
        return accession.split('|')[1]
    else:
        return 'unknown'
    
ori_df['Entry'] = ori_df['fasta_header'].apply(pull_acc)

# Remove the contamination and the incorrectly aligned entries
ori_df = ori_df[ori_df.Entry != 'cont']
ori_df = ori_df[ori_df.Entry != 'REV']

ori_df

,fasta_header,BL11,BL12,BL21,BL22,CF11,CF12,CF21,CF22,POOL1,...,POOL4,S111,S112,S121,S122,S711,S712,S721,S722,Entry
6,EXTRA_0117 (+1),1.440225e+07,1.976366e+07,1.777267e+07,1.323484e+07,1.061200e+07,1.441863e+07,1.598302e+07,1.290840e+07,3.680239e+07,...,3.689753e+07,1.271964e+07,1.192148e+07,1.637461e+07,1.248008e+07,1.228182e+08,1.244550e+08,1.007215e+08,1.100053e+08,extra
7,EXTRA_0118,4.354697e+08,4.574848e+08,4.548247e+08,4.164842e+08,3.489221e+08,4.174575e+08,3.975685e+08,3.708042e+08,8.357021e+08,...,8.453671e+08,1.744299e+09,1.499142e+09,1.547300e+09,1.508134e+09,1.096133e+09,1.057119e+09,1.002331e+09,1.096688e+09,extra
8,EXTRA_0119,1.539789e+08,2.000820e+08,1.979865e+08,1.596371e+08,1.080412e+08,2.495407e+08,1.974197e+08,1.836982e+08,8.359798e+08,...,8.412190e+08,2.061747e+09,1.934594e+09,1.918482e+09,1.835840e+09,1.233054e+09,1.280986e+09,1.098762e+09,1.286722e+09,extra
9,EXTRA_0121,1.500844e+07,1.761701e+07,1.691416e+07,1.469569e+07,1.207333e+07,1.455480e+07,1.614900e+07,1.252156e+07,3.134059e+07,...,3.136487e+07,1.166578e+07,1.177868e+07,1.510703e+07,1.237014e+07,8.915036e+07,9.935692e+07,8.253212e+07,8.880221e+07,extra
10,EXTRA_0125,3.332058e+07,3.612639e+07,3.704711e+07,3.125256e+07,5.372157e+07,5.407940e+07,5.730539e+07,5.168573e+07,6.714455e+07,...,6.536687e+07,3.140472e+07,3.339494e+07,3.510481e+07,3.100372e+07,1.433261e+08,1.434790e+08,1.489743e+08,1.757447e+08,extra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907,tr|A0A140SSC5|A0A140SSC5_ECOBD,2.947686e+06,1.996774e+06,2.225339e+06,2.449551e+06,2.298617e+06,2.015096e+06,1.811964e+06,1.979171e+06,1.999134e+06,...,1.950416e+06,1.751082e+06,1.652027e+06,1.641154e+06,1.746556e+06,1.701948e+06,1.677219e+06,1.916319e+06,1.604518e+06,A0A140SSC5
1908,tr|C6EGE8|C6EGE8_ECOBD,3.293396e+07,2.955156e+07,2.838886e+07,2.649919e+07,3.626575e+07,3.445293e+07,3.259728e+07,3.229829e+07,3.380977e+07,...,3.344518e+07,4.331682e+07,3.605674e+07,3.886754e+07,3.496004e+07,3.630116e+07,2.949580e+07,3.714245e+07,3.704132e+07,C6EGE8
1914,tr|A0A140N3T4|A0A140N3T4_ECOBD_family,9.913333e+07,1.185221e+08,1.178427e+08,8.869173e+07,8.172502e+07,9.163873e+07,1.089065e+08,7.856521e+07,2.432978e+08,...,2.408500e+08,7.316567e+07,6.734767e+07,9.699063e+07,7.824137e+07,7.371883e+08,7.150568e+08,7.386923e+08,8.135044e+08,A0A140N3T4
1915,EXTRA_0123_family,9.621162e+07,1.097802e+08,1.083865e+08,9.617083e+07,8.786867e+07,9.855935e+07,1.051171e+08,8.943943e+07,1.581136e+08,...,1.573007e+08,7.988909e+07,8.524818e+07,8.753795e+07,8.202545e+07,3.717960e+08,3.870587e+08,3.546405e+08,3.905648e+08,extra


### Map the names of the expressed proteins to their intensities

In [11]:
def expressed_protein_accession(row):
    entry = row['fasta_header']
    for string in exp_df['Entry']:
        if string in entry:
            return exp_accession[string]
    return row['Entry']

def find_expressed(row):
    entry = row['fasta_header']
    for string in exp_df['Entry']:
        if string in entry:
            print('found')
            return 'exp'
    return 'not_exp'

exp_df = pd.read_csv('../Initial files/expressed_protein_list.csv')
exp_entry = exp_df['Entry'].tolist()

exp_names = dict(zip(exp_df['Entry'],exp_df['Protein name']))
exp_accession = dict(zip(exp_df['Entry'],exp_df['Accession']))


ori_df['Entry'] = ori_df.apply(expressed_protein_accession, axis=1)

['EXTRA_0117', 'EXTRA_0118', 'EXTRA_0119', 'EXTRA_0120', 'EXTRA_0121', 'EXTRA_0122', 'EXTRA_0123', 'EXTRA_0124', 'EXTRA_0125', 'EXTRA_0126', 'EXTRA_0127', 'EXTRA_0129', 'EXTRA_0130']


,fasta_header,BL11,BL12,BL21,BL22,CF11,CF12,CF21,CF22,POOL1,...,POOL4,S111,S112,S121,S122,S711,S712,S721,S722,Entry
1001,tr|A0A140NAI0|A0A140NAI0_ECOBD,3.971636e+06,4.043056e+06,3.957685e+06,3.803254e+06,4.245724e+06,4.525364e+06,4.670624e+06,4.373359e+06,5.170285e+06,...,5.167611e+06,3.441049e+06,3.612092e+06,3.553447e+06,3.438511e+06,8.250028e+06,8.614739e+06,1.016302e+07,1.044174e+07,A0A140NAI0


### Join the protein information with the normalized intensity data

In [12]:
ref_df = pd.read_csv('../Initial files/proteome_UP000002032_info.csv',index_col=False)

# Join the information from the proteome with the mass spectrometry data
combined_df = ori_df.set_index('Entry').join(ref_df.set_index('Entry'))


### Identify the proteins that were intentionally over-expressed

In [13]:
def expressed_protein_name(row):
    entry = row['fasta_header']
    for string in exp_df['Entry']:
        if string in entry:
            return exp_names[string]
    return row['Protein_names']

def find_expressed(name):
    if name in exp_prots:
        return 'exp'
    else:
        return 'not_exp'
    
exp_prots = exp_df['Protein name'].tolist()

combined_df['Protein_names'] = combined_df.apply(expressed_protein_name,axis=1)
combined_df['expressed'] = combined_df['Protein_names'].apply(find_expressed)



0     EXTRA_0117
1     EXTRA_0118
2     EXTRA_0119
3     EXTRA_0120
4     EXTRA_0121
5     EXTRA_0122
6     EXTRA_0123
7     EXTRA_0124
8     EXTRA_0125
9     EXTRA_0126
10    EXTRA_0127
11    EXTRA_0129
12    EXTRA_0130
Name: Entry, dtype: object
EXTRA_0117 Peptide chain release factor 3 (RF-3)
EXTRA_0118 Elongation factor Tu (EF-Tu)
EXTRA_0119 Elongation factor Ts (EF-Ts)
EXTRA_0121 Elongation factor 4 (EF-4) (EC 3.6.5.n1) (Ribosomal back-translocase LepA)
EXTRA_0125 Translation initiation factor IF-3
EXTRA_0126 Elongation factor G (EF-G)
EXTRA_0127 Translation initiation factor IF-1
EXTRA_0130 Cyan Fluorescent Protein
EXTRA_0123 Alanine--tRNA ligase (EC 6.1.1.7) (Alanyl-tRNA synthetase) (AlaRS)
Peptide chain release factor 3 (RF-3)
Elongation factor Tu (EF-Tu)
Elongation factor Ts (EF-Ts)
Elongation factor 4 (EF-4) (EC 3.6.5.n1) (Ribosomal back-translocase LepA)
Translation initiation factor IF-3
Elongation factor G (EF-G)
Translation initiation factor IF-1
Cyan Fluorescent Protein


,fasta_header,BL11,BL12,BL21,BL22,CF11,CF12,CF21,CF22,POOL1,...,Gene_ontology_cellular_component,Gene_ontology_GO,Gene_ontology_molecular_function,Gene_ontology_IDs,EC_number,Intramembrane,Subcellular_location_CC,Topological_domain,Transmembrane,expressed
Entry,,,,,,,,,,,,,,,,,,,,,
A0A140N3T4,tr|A0A140N3T4|A0A140N3T4_ECOBD_family,9.913333e+07,1.185221e+08,1.178427e+08,8.869173e+07,8.172502e+07,9.163873e+07,1.089065e+08,7.856521e+07,2.432978e+08,...,cytoplasm [GO:0005737],cytoplasm [GO:0005737]; GTP binding [GO:000552...,GTPase activity [GO:0003924]; GTP binding [GO:...,GO:0003743; GO:0003924; GO:0005525; GO:0005737,NaN,NaN,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|H...,NaN,NaN,exp


### Calculate comparisons between replicates and samples
##### These calculations will be used to generate the volcano plots in the R Jupyter Notebook titled "Extract comparison volcano plots"

In [14]:
def mean_calc(row, extracts=None):
    return np.mean([row[ex] for ex in extracts])

def std_calc(row,extracts=None):
    return np.std([row[ex] for ex in extracts])

def ttest_ind(row, extracts=None):
    '''
    The t-test used in this funnction is a two-tailed t-test with the 
    null hypothesis that the two sample means will be the same
    '''
    ex1_names = [extracts[0]+name for name in replicates]
    ex2_names = [extracts[1]+name for name in replicates]

    ex1 = [row[ex] for ex in ex1_names]
    ex2 = [row[ex] for ex in ex2_names]

    return stats.ttest_ind(ex1,ex2)[1]
    
extracts = ['BL','CF','S1','S7']
replicates = ['11','12','21','22']

for ex in extracts:
    ex_names = [ex+name for name in replicates]
       
    col_name = ex + '_mean'
    combined_df[col_name] = combined_df.apply(mean_calc, axis=1, extracts = ex_names)
    
    col_name = ex + '_std'
    combined_df[col_name] = combined_df.apply(std_calc, axis=1, extracts = ex_names)
    
pairs = [['CF','BL'],['S1','BL'],['S7','BL']]

for pair in pairs:
    diff_col = pair[0]+'_'+pair[1]+'_fc'
    combined_df[diff_col] = combined_df[pair[0]+'_mean']/combined_df[pair[1]+'_mean']
    ttest_col = pair[0]+'_'+pair[1]+'_p_val'
    combined_df[ttest_col] = combined_df.apply(ttest_ind, axis=1, extracts = pair)
combined_df

,fasta_header,BL11,BL12,BL21,BL22,CF11,CF12,CF21,CF22,POOL1,...,S1_mean,S1_std,S7_mean,S7_std,CF_BL_fc,CF_BL_p_val,S1_BL_fc,S1_BL_p_val,S7_BL_fc,S7_BL_p_val
Entry,,,,,,,,,,,,,,,,,,,,,
A0A140NC65,EXTRA_0117 (+1),1.440225e+07,1.976366e+07,1.777267e+07,1.323484e+07,1.061200e+07,1.441863e+07,1.598302e+07,1.290840e+07,3.680239e+07,...,1.337396e+07,1.756471e+06,1.145000e+08,9.725529e+06,0.827362,0.187226,0.820823,1.587303e-01,7.027404,2.749047e-06
A0A140N6W0,EXTRA_0118,4.354697e+08,4.574848e+08,4.548247e+08,4.164842e+08,3.489221e+08,4.174575e+08,3.975685e+08,3.708042e+08,8.357021e+08,...,1.574719e+09,9.956742e+07,1.063068e+09,3.856144e+07,0.869911,0.018059,3.570258,1.194812e-06,2.410225,2.300600e-07
A0A140NFD7,EXTRA_0119,1.539789e+08,2.000820e+08,1.979865e+08,1.596371e+08,1.080412e+08,2.495407e+08,1.974197e+08,1.836982e+08,8.359798e+08,...,1.937665e+09,8.084256e+07,1.224881e+09,7.573781e+07,1.037960,0.838248,10.890588,2.836386e-08,6.884404,4.364187e-07
A0A140N6C6,EXTRA_0121,1.500844e+07,1.761701e+07,1.691416e+07,1.469569e+07,1.207333e+07,1.455480e+07,1.614900e+07,1.252156e+07,3.134059e+07,...,1.273041e+07,1.397976e+06,8.996040e+07,6.030578e+06,0.860877,0.108104,0.792736,2.143383e-02,5.601930,8.056661e-07
A0A140N9R4,EXTRA_0125,3.332058e+07,3.612639e+07,3.704711e+07,3.125256e+07,5.372157e+07,5.407940e+07,5.730539e+07,5.168573e+07,6.714455e+07,...,3.272705e+07,1.644554e+06,1.528810e+08,1.339503e+07,1.573847,0.000030,0.950355,3.345745e-01,4.439484,5.327811e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A0A140SSC5,tr|A0A140SSC5|A0A140SSC5_ECOBD,2.947686e+06,1.996774e+06,2.225339e+06,2.449551e+06,2.298617e+06,2.015096e+06,1.811964e+06,1.979171e+06,1.999134e+06,...,1.697705e+06,5.128378e+04,1.725001e+06,1.161177e+05,0.842557,0.146237,0.705954,1.373626e-02,0.717304,1.913908e-02
C6EGE8,tr|C6EGE8|C6EGE8_ECOBD,3.293396e+07,2.955156e+07,2.838886e+07,2.649919e+07,3.626575e+07,3.445293e+07,3.259728e+07,3.229829e+07,3.380977e+07,...,3.830028e+07,3.227930e+06,3.499518e+07,3.191639e+06,1.155407,0.031656,1.305244,8.074961e-03,1.192609,4.826855e-02
A0A140N3T4,tr|A0A140N3T4|A0A140N3T4_ECOBD_family,9.913333e+07,1.185221e+08,1.178427e+08,8.869173e+07,8.172502e+07,9.163873e+07,1.089065e+08,7.856521e+07,2.432978e+08,...,7.893634e+07,1.111348e+07,7.511105e+08,3.721860e+07,0.850646,0.164757,0.744349,3.181924e-02,7.082776,1.257925e-07


In [15]:
combined_df.to_csv('../Intermediate data/labeled_IRS_proteins_exp.csv')